In [61]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer


In [62]:
train_data = pd.read_csv("C:\\Users\\kalin\\OneDrive - UW-Eau Claire\\Schoolwork\\FakeNewsResearch\\Corona_NLP_test.csv")

In [63]:
train_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [64]:
label_mapping={'Extremely Negative':0,'Negative':0,'Neutral':1,'Positive':2,'Extremely Positive':2}
#True: True, Mostly-True
#Half-True: Half-True
#False: False, Barely-True

#New Labels: True-0, False-1, Half-True-2

#label_mapping=df['Labels']
df['Sentiment'] = df['Sentiment'].map(label_mapping)
print(df['Sentiment'].unique())

[0 2 1]


In [65]:
stop=set(stopwords.words('english'))
train_data["OriginalTweet"]=train_data["OriginalTweet"].apply(lambda x:' '.join([word for word in x.split() if word.lower() not in stop]))

In [66]:
df=pd.DataFrame(train_data,columns=['OriginalTweet','Sentiment'])
df.head()

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,"find hand sanitizer Fred Meyer, turned #Amazon...",Positive
2,Find protect loved ones #coronavirus. ?,Extremely Positive
3,#Panic buying hits #NewYork City anxious shopp...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [67]:
tokenizer = RegexpTokenizer(r"[\w']+")
df["tokenizedTweet"]=df["OriginalTweet"].map(tokenizer.tokenize)

In [68]:
PorterStemmer=PorterStemmer()
df["PorterStemmer"] = df["tokenizedTweet"].apply(lambda x : [PorterStemmer.stem(word) for word in x])

In [69]:
df.head()

,OriginalTweet,Sentiment,tokenizedTweet,PorterStemmer
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,"[TRENDING, New, Yorkers, encounter, empty, sup...","[trend, new, yorker, encount, empti, supermark..."
1,"find hand sanitizer Fred Meyer, turned #Amazon...",Positive,"[find, hand, sanitizer, Fred, Meyer, turned, A...","[find, hand, sanit, fred, meyer, turn, amazon,..."
2,Find protect loved ones #coronavirus. ?,Extremely Positive,"[Find, protect, loved, ones, coronavirus]","[find, protect, love, one, coronaviru]"
3,#Panic buying hits #NewYork City anxious shopp...,Negative,"[Panic, buying, hits, NewYork, City, anxious, ...","[panic, buy, hit, newyork, citi, anxiou, shopp..."
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,"[toiletpaper, dunnypaper, coronavirus, coronav...","[toiletpap, dunnypap, coronaviru, coronavirusa..."


In [70]:
df["PorterStemmerString"]=df["PorterStemmer"].apply(lambda x:' '.join(x))

In [71]:
from sklearn.model_selection import train_test_split


X_train,X_test, y_train, y_test = train_test_split(df['PorterStemmerString'],df['Sentiment'],test_size=.2,random_state=6)

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
all_text = pd.concat([X_train,X_test]) 
tfidf_vectorizer = TfidfVectorizer()
# Fit the vectorizer on the entire dataset

X_train_tfidf=tfidf_vectorizer.fit_transform(X_train)

#Transform training and test data using the same vectorizer
#X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [73]:
import xgboost as xgb

estimator_xgb = xgb.XGBClassifier(objective='multi:softmax')
#estimator_xgb = xgb.XGBClassifier()
estimator_xgb.fit(X_train_tfidf, y_train)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4], got ['Extremely Negative' 'Extremely Positive' 'Negative' 'Neutral' 'Positive']